In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from pandas import json_normalize
from requests.exceptions import HTTPError
from requests.auth import HTTPBasicAuth
from getpass import getpass
pd.set_option('display.max_rows', None)

In [2]:
from clean_taxi import df

In [3]:
df.index.rename('Transaction Number', inplace=True)

In [4]:
df = df.rename(columns={'vendorid': 'Vendor Id'
                   ,'tpep_pickup_datetime':'Pickup Datetime'
                   ,'tpep_dropoff_datetime': 'Dropoff Datetime'
                   ,'passenger_count':'Passenger Count'
                   , 'trip_distance': 'Trip Distance'
                   , 'ratecodeid': 'Rate Code Id'
                   , 'store_and_fwd_flag':'Store and Forward'
                   , 'pulocationid': 'Pick Up Location Id'
                   ,'dolocationid': 'Drop Off Location Id'
                   ,'payment_type': 'Payment Type'
                   , 'fare_amount': 'Fare Amount'
                   , 'extra': 'Extra'
                   ,'mta_tax': 'MTA Tax'
                   , 'tip_amount': 'Tip Amount'
                   , 'tolls_amount': 'Tolls Amount'
                   , 'improvement_surcharge': 'Improvement Surcharge'
                   ,'total_amount': 'Total Amount'
                   , 'congestion_surcharge':'Congestion Surcharge'})
                   
# df.head()        



In [5]:
df.columns

Index(['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
       'Trip Distance', 'Rate Code Id', 'Store and Forward',
       'Pick Up Location Id', 'Drop Off Location Id', 'Payment Type',
       'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount',
       'Improvement Surcharge', 'Total Amount', 'Congestion Surcharge'],
      dtype='object')

In [6]:
cols = ['Pick Up Location Id', 'Drop Off Location Id', 'Passenger Count', 'Trip Distance', 
        'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount',
       'Improvement Surcharge', 'Total Amount', 'Congestion Surcharge']

for col in cols:
    df[col] = pd.to_numeric(df[col])

In [7]:
# datetimes are datetimes
df['Pickup Datetime'] = pd.to_datetime(df['Pickup Datetime'])

In [8]:
# datetimes are datetimes
df['Dropoff Datetime'] = pd.to_datetime(df['Dropoff Datetime'])

In [9]:
df['Trip Time (hrs)']=((df['Dropoff Datetime'] - df['Pickup Datetime'])/np.timedelta64(1, 'h')).round(2)

In [10]:
# engineer feature: mph (time / distance )
df['Trip Speed mph'] = (df['Trip Distance'] / df['Trip Time (hrs)']).round(2)


In [11]:
# replace 0 with "None", 0.5 with "Rush Hour"
# df['Extra'] = df['Extra'].replace({'0.5':'Rush Hour', '1': 'Overnight', '0': 'None'})

In [12]:
df['Rate Code Id'] = df['Rate Code Id'].replace({'1':'Standard rate' 
                                                 , '2':'JFK', 3:'Newark'
                                                 , '4':'Nassau or Westchester'
                                                 , '5':'Negotiated fare'
                                                 , '6':'Group ride'})

In [13]:
df['Store and Forward'] = df['Store and Forward'].replace({'Y':True, 'N':False})

In [14]:
df['Payment Type'] = df['Payment Type'].replace({'1':'Credit Card'
                                                 , '2':'Cash'
                                                 , '3':'No Charge'
                                                 , '4':'Dispute'
                                                 , '5':'Unknown'
                                                 , '6':'Voided_Trip'})

In [15]:
df['Vendor Id'] = df['Vendor Id'].replace({'1':'Creative Mobole'
                                           , '2':'VeriFone'})

In [16]:
df.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Rate Code Id,Store and Forward,Pick Up Location Id,Drop Off Location Id,Payment Type,Fare Amount,Extra,MTA Tax,Tip Amount,Tolls Amount,Improvement Surcharge,Total Amount,Congestion Surcharge,Trip Time (hrs),Trip Speed mph
Transaction Number,,,,,,,,,,,,,,,,,,,,
0,Creative Mobole,2019-03-28 21:47:41,2019-03-28 22:05:29,1,3.60,Standard rate,False,164,231,Credit Card,15.0,3.0,0.5,3.75,0.0,0.3,22.55,2.5,0.30,12.00
1,VeriFone,2019-03-28 20:56:55,2019-03-28 21:05:48,6,1.49,Standard rate,False,211,234,Credit Card,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5,0.15,9.93
2,VeriFone,2019-03-28 21:06:47,2019-03-28 21:23:33,6,2.98,Standard rate,False,234,237,Credit Card,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,0.28,10.64
3,Creative Mobole,2019-03-28 21:41:51,2019-03-28 21:50:20,1,0.70,Standard rate,False,48,100,Credit Card,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5,0.14,5.00
4,Creative Mobole,2019-03-28 21:52:05,2019-03-28 21:58:11,1,0.90,Standard rate,False,100,234,Cash,6.0,3.0,0.5,0.00,0.0,0.3,9.80,2.5,0.10,9.00


In [17]:
df = pd.get_dummies(df, columns=['Payment Type'], dummy_na=True, drop_first=False)

In [18]:
df.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Rate Code Id,Store and Forward,Pick Up Location Id,Drop Off Location Id,Fare Amount,Extra,MTA Tax,Tip Amount,Tolls Amount,Improvement Surcharge,Total Amount,Congestion Surcharge,Trip Time (hrs),Trip Speed mph,Payment Type_Cash,Payment Type_Credit Card,Payment Type_Dispute,Payment Type_No Charge,Payment Type_nan
Transaction Number,,,,,,,,,,,,,,,,,,,,,,,,
0,Creative Mobole,2019-03-28 21:47:41,2019-03-28 22:05:29,1,3.60,Standard rate,False,164,231,15.0,3.0,0.5,3.75,0.0,0.3,22.55,2.5,0.30,12.00,0,1,0,0,0
1,VeriFone,2019-03-28 20:56:55,2019-03-28 21:05:48,6,1.49,Standard rate,False,211,234,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5,0.15,9.93,0,1,0,0,0
2,VeriFone,2019-03-28 21:06:47,2019-03-28 21:23:33,6,2.98,Standard rate,False,234,237,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,0.28,10.64,0,1,0,0,0
3,Creative Mobole,2019-03-28 21:41:51,2019-03-28 21:50:20,1,0.70,Standard rate,False,48,100,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5,0.14,5.00,0,1,0,0,0
4,Creative Mobole,2019-03-28 21:52:05,2019-03-28 21:58:11,1,0.90,Standard rate,False,100,234,6.0,3.0,0.5,0.00,0.0,0.3,9.80,2.5,0.10,9.00,1,0,0,0,0


### Did
- reindex so each row is a trip instance CHECK
- datetimes are datetimes CHECK
- engineer feature: mph (time / distance ) CHECK
- Rename extra to none, overnight, rush hour CHECK 
- convert all numeric strings to floats/numerics CHECK
- match taxizone with zipcode or location name CHECK
- Convert Rate Code id to 1= Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester  5=Negotiated fare, 6=Group ride
- one hot encode the payment types 

### To Do
- Rename and reorder the taxicode pickup and drop off names


In [19]:
taxi_zones = pd.read_csv('taxi+_zone_lookup.csv')


In [20]:
#left join df wtih taxi_zones on df[Pick Up Location Id]== taxi_zones[LocationId] 
# and df[Drop Off Location ID] == taxi_zone[LocationID]
taxi_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [21]:
df = df.merge(taxi_zones, how='left', left_on='Pick Up Location Id',  right_on='LocationID', suffixes=('_Pick_Up', '_Pick_Up_Zone_name'))

In [22]:
df = df.drop('LocationID', axis=1)

In [23]:
df = df.merge(taxi_zones, how='left', left_on='Drop Off Location Id',  right_on='LocationID', suffixes=('_PickUp', '_DropOff'))

In [24]:
df = df.drop('LocationID', axis=1)

In [25]:
df.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Rate Code Id,Store and Forward,Pick Up Location Id,Drop Off Location Id,Fare Amount,Extra,MTA Tax,Tip Amount,Tolls Amount,Improvement Surcharge,Total Amount,Congestion Surcharge,Trip Time (hrs),Trip Speed mph,Payment Type_Cash,Payment Type_Credit Card,Payment Type_Dispute,Payment Type_No Charge,Payment Type_nan,Borough_PickUp,Zone_PickUp,service_zone_PickUp,Borough_DropOff,Zone_DropOff,service_zone_DropOff
0,Creative Mobole,2019-03-28 21:47:41,2019-03-28 22:05:29,1,3.60,Standard rate,False,164,231,15.0,3.0,0.5,3.75,0.0,0.3,22.55,2.5,0.30,12.00,0,1,0,0,0,Manhattan,Midtown South,Yellow Zone,Manhattan,TriBeCa/Civic Center,Yellow Zone
1,VeriFone,2019-03-28 20:56:55,2019-03-28 21:05:48,6,1.49,Standard rate,False,211,234,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5,0.15,9.93,0,1,0,0,0,Manhattan,SoHo,Yellow Zone,Manhattan,Union Sq,Yellow Zone
2,VeriFone,2019-03-28 21:06:47,2019-03-28 21:23:33,6,2.98,Standard rate,False,234,237,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,0.28,10.64,0,1,0,0,0,Manhattan,Union Sq,Yellow Zone,Manhattan,Upper East Side South,Yellow Zone
3,Creative Mobole,2019-03-28 21:41:51,2019-03-28 21:50:20,1,0.70,Standard rate,False,48,100,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5,0.14,5.00,0,1,0,0,0,Manhattan,Clinton East,Yellow Zone,Manhattan,Garment District,Yellow Zone
4,Creative Mobole,2019-03-28 21:52:05,2019-03-28 21:58:11,1,0.90,Standard rate,False,100,234,6.0,3.0,0.5,0.00,0.0,0.3,9.80,2.5,0.10,9.00,1,0,0,0,0,Manhattan,Garment District,Yellow Zone,Manhattan,Union Sq,Yellow Zone


In [26]:
df.columns

Index(['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
       'Trip Distance', 'Rate Code Id', 'Store and Forward',
       'Pick Up Location Id', 'Drop Off Location Id', 'Fare Amount', 'Extra',
       'MTA Tax', 'Tip Amount', 'Tolls Amount', 'Improvement Surcharge',
       'Total Amount', 'Congestion Surcharge', 'Trip Time (hrs)',
       'Trip Speed mph', 'Payment Type_Cash', 'Payment Type_Credit Card',
       'Payment Type_Dispute', 'Payment Type_No Charge', 'Payment Type_nan',
       'Borough_PickUp', 'Zone_PickUp', 'service_zone_PickUp',
       'Borough_DropOff', 'Zone_DropOff', 'service_zone_DropOff'],
      dtype='object')

In [27]:
reordered_columns = ['Vendor Id', 'Pickup Datetime', 'Dropoff Datetime', 'Passenger Count',
                'Trip Distance',  'Trip Time (hrs)', 'Trip Speed mph',
                'Rate Code Id', 'Store and Forward',
                'Pick Up Location Id', 'Borough_PickUp', 'Zone_PickUp', 'service_zone_PickUp',
                'Drop Off Location Id', 'Borough_DropOff', 'Zone_DropOff', 'service_zone_DropOff', 
                'Fare Amount', 'Extra', 'MTA Tax', 'Tip Amount', 'Tolls Amount', 'Improvement Surcharge',
                'Total Amount', 'Congestion Surcharge', 'Payment Type_Cash', 'Payment Type_Credit Card',
                'Payment Type_No Charge', 'Payment Type_nan', 'Payment Type_Dispute']
       
       


In [28]:
df = df.reindex(columns=reordered_columns)

In [29]:
df.head()

,Vendor Id,Pickup Datetime,Dropoff Datetime,Passenger Count,Trip Distance,Trip Time (hrs),Trip Speed mph,Rate Code Id,Store and Forward,Pick Up Location Id,Borough_PickUp,Zone_PickUp,service_zone_PickUp,Drop Off Location Id,Borough_DropOff,Zone_DropOff,service_zone_DropOff,Fare Amount,Extra,MTA Tax,Tip Amount,Tolls Amount,Improvement Surcharge,Total Amount,Congestion Surcharge,Payment Type_Cash,Payment Type_Credit Card,Payment Type_No Charge,Payment Type_nan,Payment Type_Dispute
0,Creative Mobole,2019-03-28 21:47:41,2019-03-28 22:05:29,1,3.60,0.30,12.00,Standard rate,False,164,Manhattan,Midtown South,Yellow Zone,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,15.0,3.0,0.5,3.75,0.0,0.3,22.55,2.5,0,1,0,0,0
1,VeriFone,2019-03-28 20:56:55,2019-03-28 21:05:48,6,1.49,0.15,9.93,Standard rate,False,211,Manhattan,SoHo,Yellow Zone,234,Manhattan,Union Sq,Yellow Zone,7.5,0.5,0.5,2.26,0.0,0.3,13.56,2.5,0,1,0,0,0
2,VeriFone,2019-03-28 21:06:47,2019-03-28 21:23:33,6,2.98,0.28,10.64,Standard rate,False,234,Manhattan,Union Sq,Yellow Zone,237,Manhattan,Upper East Side South,Yellow Zone,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,0,1,0,0,0
3,Creative Mobole,2019-03-28 21:41:51,2019-03-28 21:50:20,1,0.70,0.14,5.00,Standard rate,False,48,Manhattan,Clinton East,Yellow Zone,100,Manhattan,Garment District,Yellow Zone,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5,0,1,0,0,0
4,Creative Mobole,2019-03-28 21:52:05,2019-03-28 21:58:11,1,0.90,0.10,9.00,Standard rate,False,100,Manhattan,Garment District,Yellow Zone,234,Manhattan,Union Sq,Yellow Zone,6.0,3.0,0.5,0.00,0.0,0.3,9.80,2.5,1,0,0,0,0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Vendor Id                 1000 non-null   object        
 1   Pickup Datetime           1000 non-null   datetime64[ns]
 2   Dropoff Datetime          1000 non-null   datetime64[ns]
 3   Passenger Count           1000 non-null   int64         
 4   Trip Distance             1000 non-null   float64       
 5   Trip Time (hrs)           1000 non-null   float64       
 6   Trip Speed mph            998 non-null    float64       
 7   Rate Code Id              1000 non-null   object        
 8   Store and Forward         1000 non-null   bool          
 9   Pick Up Location Id       1000 non-null   int64         
 10  Borough_PickUp            1000 non-null   object        
 11  Zone_PickUp               998 non-null    object        
 12  service_zone_PickUp  